# Data EDA

* 데이터 라벨

|0|1|2|3|4|5|6|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|생활문화|스포츠|사회|세계|IT과학|경제|정치|

# Library import 단계

In [1]:
import transformers
import torch
import pandas as pd
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_huggingface.llms import HuggingFacePipeline
from tqdm import tqdm

tqdm.pandas()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Noise vs Non - Noise 분류
- Ko-Llama3 사용

In [2]:
# train_data 불러오기
data = pd.read_csv('./data/train.csv')
data.head()

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6


- 초기 실행 시 10m 13.4s 소요(m2max, ram 32GB 기준). 
- 재실행시 37~8s
- 모델 용량 4.89 + 4.92 + 5.00 + 1.55GB

In [ ]:
model_id = "saltlux/Ko-Llama3-Luxia-8B"

pipeline = transformers.pipeline(
    "text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype": torch.bfloat16}, 
    device_map="auto",
    max_new_tokens=50,
)
pipeline("<|begin_of_text|>안녕하세요. 솔트룩스 AI Labs 입니다.")

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
prompt_template = """
다음은 한국어 뉴스 제목입니다:
"{text}"
이 제목이 노이즈가 포함되어 있다면 'Yes', 그렇지 않다면 'No'로 답변하세요.
"""

Ko-Llama3 사용 텍스트 노이즈 감지
- 소요 시간 2 ~ 3 시간. 1 iter 당 3.4 ~ 3.5 소요

In [ ]:
def detect_noise(df: pd.DataFrame) -> pd.DataFrame:
    descriptions = []
    for text in tqdm(df['text'], desc="Processing"):
        prompt = prompt_template.format(text=text)
        response = llm.invoke(prompt)
        if 'Yes' in response:
            descriptions.append('Yes')
        elif 'No' in response:
            descriptions.append('No')
        else:
            descriptions.append('Uncertain')       
    df['description'] = descriptions
    return df


noise_detected_data = detect_noise(data)
noise_detected_data

In [ ]:
noise_data_count = len(data[data['is_noisy'] == True])
noise_data_count